In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, concatenate, Input
from tensorflow.keras.models import Model
import os
import numpy as np
import cv2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
#1. Define the UNet Model
# -------------------------
def unet_model(input_size=(224, 224, 3)):
    inputs = Input(input_size)
    
    # Encoder (Contracting Path)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D(pool_size=(2, 2))(c1)
    
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D(pool_size=(2, 2))(c2)
    
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D(pool_size=(2, 2))(c3)
    
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
    
    # Bottleneck
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
    
    # Decoder (Expanding Path)
    u6 = UpSampling2D(size=(2, 2))(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
    
    u7 = UpSampling2D(size=(2, 2))(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
    
    u8 = UpSampling2D(size=(2, 2))(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
    
    u9 = UpSampling2D(size=(2, 2))(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [3]:
# 2. Data Loading and Preprocessing
# -------------------------------
def load_data(image_dir, img_size=(224, 224)):
    """
    Loads images from the given directory and creates dummy segmentation masks.
    For demonstration:
      - If category is 'healthy' (case insensitive), the mask is all zeros.
      - Otherwise, the mask is all ones.
    """
    images = []
    masks = []
    categories = os.listdir(image_dir)
    
    for category in categories:
        category_path = os.path.join(image_dir, category)
        if os.path.isdir(category_path):
            for file in os.listdir(category_path):
                img_path = os.path.join(category_path, file)
                img = cv2.imread(img_path)
                if img is None:
                    continue
                img = cv2.resize(img, img_size)
                img = img / 255.0  # Normalize image
                images.append(img)
                # Create dummy mask
                if category.lower() == 'healthy':
                    mask = np.zeros((img_size[0], img_size[1], 1), dtype=np.float32)
                else:
                    mask = np.ones((img_size[0], img_size[1], 1), dtype=np.float32)
                masks.append(mask)
    return np.array(images), np.array(masks)

# Update this path to point to your dataset directory
image_dir = "/kaggle/input/thermal-images-diseased-healthy-leaves-paddy/thermal images UL"
X, y = load_data(image_dir)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
# 3. Data Augmentation for Segmentation (Images & Masks)
# -----------------------------------------------------
# To ensure the same random transformation is applied to images and masks,
# we use two ImageDataGenerators with the same seed.
seed = 42
batch_size = 16

image_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

mask_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Fit the generators (not strictly required but sometimes helps)
image_datagen.fit(X_train, augment=True, seed=seed)
mask_datagen.fit(y_train, augment=True, seed=seed)

# Create generators
generator_image = image_datagen.flow(X_train, batch_size=batch_size, seed=seed)
generator_mask = mask_datagen.flow(y_train, batch_size=batch_size, seed=seed)

def train_generator():
    while True:
        img_batch = next(generator_image)
        mask_batch = next(generator_mask)
        yield (img_batch, mask_batch)

steps_per_epoch = len(X_train) // batch_size


In [ ]:
# 4. Build, Train, and Evaluate Model
# ------------------------------
model = unet_model()
model.summary()

history = model.fit(
    train_generator(),
    steps_per_epoch=steps_per_epoch,
    validation_data=(X_val, y_val),
    epochs=20
)

# Plot training history
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Evaluate model on validation set
eval_results = model.evaluate(X_val, y_val)
print(f"Validation Loss: {eval_results[0]}, Validation Accuracy: {eval_results[1]}")


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 224, 224, 64)   │          1,792 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 224, 224, 64)   │         36,928 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 112, 112, 64)   │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 112, 112, 128)  │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 112, 112, 128)  │        147,584 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 56, 56, 128)    │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 56, 56, 256)    │        295,168 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 56, 56, 256)    │        590,080 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 28, 28, 256)    │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 28, 28, 512)    │      1,180,160 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 28, 28, 512)    │      2,359,808 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 14, 14, 512)    │              0 │ conv2d_7[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 14, 14, 1024)   │      4,719,616 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 14, 14, 1024)   │      9,438,208 │ conv2d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d             │ (None, 28, 28, 1024)   │              0 │ conv2d_9[0][0]         │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 28, 28, 1536)   │              0 │ up_sampling2d[0][0],   │
│                      

 Total params: 31,378,945 (119.70 MB)

 Trainable params: 31,378,945 (119.70 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.6948 - loss: 0.5815 - val_accuracy: 0.8438 - val_loss: 0.6610
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 32s 325ms/step - accuracy: 0.8431 - loss: 0.4822 - val_accuracy: 0.8438 - val_loss: 0.4488
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 324ms/step - accuracy: 0.8581 - loss: 0.4134 - val_accuracy: 0.8438 - val_loss: 0.4218
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 323ms/step - accuracy: 0.8469 - loss: 0.4235 - val_accuracy: 0.8438 - val_loss: 0.4245
Epoch 5/20
28/31 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step - accuracy: 0.8510 - loss: 0.4133

In [ ]:
# 5. Visualize Predictions
# ------------------------------
preds = model.predict(X_val)

n = 3  # Number of examples to display
plt.figure(figsize=(15, 10))
for i in range(n):
    # Display original image
    plt.subplot(3, n, i+1)
    plt.imshow(X_val[i])
    plt.title('Image')
    plt.axis('off')
    
    # Display ground truth mask
    plt.subplot(3, n, i+n+1)
    plt.imshow(y_val[i].squeeze(), cmap='gray')
    plt.title('Ground Truth')
    plt.axis('off')
    
    # Display predicted mask
    plt.subplot(3, n, i+2*n+1)
    plt.imshow(preds[i].squeeze(), cmap='gray')
    plt.title('Prediction')
    plt.axis('off')
plt.tight_layout()
plt.show()